In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import f1_score, recall_score
import os
import random, math
from sklearn.model_selection import train_test_split as sk_train_test_split
import seaborn as sns
from sklearn.metrics import accuracy_score


In [ ]:
data= pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')

In [ ]:
data.head(4)

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
X= data.drop('Outcome', axis=1)
y = data['Outcome']

In [ ]:
data['Outcome'].value_counts()


**Corralation plot**

In [ ]:
dataplot = sns.heatmap(data.corr(), cmap="YlGnBu", annot=True)

In [ ]:
# X.drop('SkinThickness',inplace= True, axis=1)

**Train_test split:**

In [ ]:
X_train,X_test, y_train, y_test =train_test_split(X,y,test_size=0.2, random_state=42) 

**Applying SMOTE algorithm:**

Be noted that only the train set should be manipulated using SMOTE algorithm. Since the test set must be representative of real-world data, we should not mix it with unreal data so that the performance of models will be evaluated on real data. 

In [ ]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_T, y_T = oversample.fit_resample(X_train, y_train)

**XGB Classifier implemenation**

In [ ]:

model = xgb.XGBClassifier(n_estimators= 350,objective= 'binary:logistic', seed=42)
model.fit(X_T, y_T )



In [ ]:
y_pred= model.predict(X_test)

In [ ]:
print(accuracy_score(y_test, y_pred))


In [ ]:
print(f1_score(y_test, y_pred))

**MLP model implementation**

In [ ]:
from tensorflow.keras.layers import Convolution1D, Dense, Flatten, GlobalAveragePooling1D, AveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, GlobalAveragePooling2D, Conv2D
from tensorflow.keras.layers import Dropout, AveragePooling2D
from sklearn.metrics import mean_squared_error

In [ ]:

n_features= X_train.shape[1]
X_T = np.array(X_T).reshape(len(X_T), n_features)
X_test= np.array(X_test).reshape(len(X_test), n_features)



In [ ]:
input_shape = (n_features,)
MLPmodel= Sequential()
MLPmodel.add(Dense(350, input_shape=input_shape, activation='relu'))
MLPmodel.add(Dropout(0.2))
MLPmodel.add(Dense(50, activation='relu'))
# MLPmodel.add(Dropout(0.1))
MLPmodel.add(Dense(1, activation='sigmoid'))

# Configure the model and start training
MLPmodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
MLPmodel.fit(X_train, y_train, epochs=20, batch_size=5, shuffle=True )

In [ ]:


prediction= MLPmodel.predict_classes(X_test)



In [ ]:
print(accuracy_score(y_test, prediction))

print(f1_score(y_test, prediction, average = 'binary'))



Submitting the results:

In [ ]:
submission = pd.DataFrame(X_test)
submission['Outcome'] = y_test
submission['Predict_Outcome'] = prediction